#Author Zichen Liu


## Motivation
Toronto is often negatively regarded in terms of its safety. Many classmates feel like it is unsafe to travel by themselves at night or even during the day. It would be beneficial to visualize the "hotpots" in Toronto, where the crime rates are the highest, so people can either avoid or be more alert when going through the area.

## Data
I am using Toronto Open Data, specifically the "Police Annual Statistical Report - Homicides". The dataset can be found here: https://open.toronto.ca/dataset/police-annual-statistical-report-homicide/

In [39]:
import pandas as pd
import re
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
from google.colab import files

uploaded = files.upload()

# Load the "Homicides" dataset
homicides = pd.read_csv('Homicides.csv')

# Function to extract coordinates using regular expressions
def extract_coordinates_regex(geometry_str):
    numbers = re.findall(r"[-+]?\d*\.\d+|\d+", geometry_str)
    if len(numbers) >= 2:
        return float(numbers[1]), float(numbers[0])  # Return as (latitude, longitude)
    else:
        return None, None

# Apply the function to extract coordinates
homicides['Latitude'], homicides['Longitude'] = zip(*homicides['geometry'].apply(extract_coordinates_regex))

# Convert OCC_DATE from Unix timestamp to datetime format
homicides['OCC_DATE'] = pd.to_datetime(homicides['OCC_DATE'], unit='ms', errors='coerce')

# Create a base map centered around Toronto
toronto_map = folium.Map(location=[43.7, -79.4], zoom_start=11)

# Create a marker cluster
marker_cluster = MarkerCluster().add_to(toronto_map)

# Add markers for each homicide incident to the marker cluster
for idx, row in homicides.iterrows():
    date_str = row['OCC_DATE'].strftime('%Y-%m-%d') if pd.notnull(row['OCC_DATE']) else 'Unknown'
    popup_label = folium.Popup(f"Homicide Type: {row['HOMICIDE_TYPE']}, Date: {date_str}", parse_html=True)
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=popup_label,
        icon=None
    ).add_to(marker_cluster)

# Create a HeatMap layer with the homicide incidents
heat_data = [[row['Latitude'], row['Longitude']] for idx, row in homicides.iterrows()]

# Create a HeatMap layer with the homicide incidents, adjusting opacity and radius
HeatMap(heat_data, gradient=None, radius=10, blur=5, max_zoom=20, max_val=10, min_opacity=0.1, max_opacity=0.5).add_to(toronto_map)

HeatMap(heat_data).add_to(toronto_map)

# Display the map
toronto_map


<ipython-input-39-471eb8787239>:46: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(heat_data, gradient=None, radius=10, blur=5, max_zoom=20, max_val=10, min_opacity=0.1, max_opacity=0.5).add_to(toronto_map)
